# 02 Forecast Model

The model is intentionally simple and stable.
The goal is a usable signal for decision-making, not maximal predictive novelty.

In [4]:
import pandas as pd
from pathlib import Path
from sklearn.linear_model import Ridge

DATA = Path("../data/processed/daily_demand.parquet")
OUT = Path("../data/processed/test_forecasts.parquet")

df = pd.read_parquet(DATA).sort_values("date").reset_index(drop=True)

# Simple, time-safe features
df["lag_1"] = df["demand"].shift(1)
df["lag_7"] = df["demand"].shift(7)
df = df.dropna().reset_index(drop=True)

split = int(len(df) * 0.8)
train = df.iloc[:split].copy()
test  = df.iloc[split:].copy()

X_train = train[["lag_1", "lag_7"]]
y_train = train["demand"]
X_test  = test[["lag_1", "lag_7"]]

model = Ridge(alpha=1.0)
model.fit(X_train, y_train)

test["forecast"] = model.predict(X_test)

# Save outputs for the decision notebook (reproducible)
test_out = test[["date", "demand", "forecast"]].copy()
test_out.to_parquet(OUT, index=False)

test_out.head()

,date,demand,forecast
578,2022-08-09,124.855261,118.555053
579,2022-08-10,119.047306,127.267636
580,2022-08-11,115.778929,122.114111
581,2022-08-12,129.078716,118.197478
582,2022-08-13,115.448616,127.164196
